In [ ]:
# Citations:
# 1) Landup, David. “Instance Segmentation with Yolov7 in Python.” Stack Abuse, Stack Abuse, 12 Oct. 2022, https://stackabuse.com/instance-segmentation-with-yolov7-in-python/. 

# 2) Kin-Yiu, W. (2022). yolov7 [Repository]. GitHub. https://github.com/WongKinYiu/yolov7
#    Wang, Chien-Yao, Alexey Bochkovskiy, and Hong-Yuan Mark Liao. "YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors." arXiv preprint arXiv:2207.02696 (2022).


In [ ]:
##########################################################
# Set the directory in which the masks will be stored
##########################################################

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
images_dir = '/content/gdrive/MyDrive/Dataset/Scenario 5/masks'

In [ ]:
##########################################################
# Clone the relevant repository, install and import the required libraries
##########################################################

In [ ]:
! git clone -b mask https://github.com/WongKinYiu/yolov7.git
! pip install pyyaml==5.1
! pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
! pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%cd yolov7
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-mask.pt -o yolov7-mask.pt

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available.")


CUDA is available.


In [ ]:
from PIL import Image
import time
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
import yaml
from torchvision import transforms
import numpy as np

from utils.datasets import letterbox
from utils.general import non_max_suppression_mask_conf

from detectron2.modeling.poolers import ROIPooler
from detectron2.structures import Boxes
from detectron2.utils.memory import retry_if_cuda_oom
from detectron2.layers import paste_masks_in_image

In [ ]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    for i in range(device_count):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")


Device 0: Tesla T4


In [ ]:



print(torch.cuda.is_available())


print(torch.cuda.current_device())


print(torch.cuda.device(0))


print(torch.cuda.device_count())


print(torch.cuda.get_device_name(0))

True
0
1
Tesla T4


In [ ]:
##########################################################
# Helper functions
##########################################################

In [ ]:

with open('data/hyp.scratch.mask.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model():
    model = torch.load('yolov7-mask.pt', map_location=device)['model']
    # Put in inference mode
    model.eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [ ]:
def run_inference(url):
    image = cv2.imread(url) # shape: (480, 640, 3)
    # print('image.shape',image.shape)
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (480, 640, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 480, 640])
    # Match tensor type (`torch.FloatTensor` -> `torch.HalfTensor`) with model
    image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 480, 640])
    output = model(image)
    return output, image


In [ ]:
def plot_results(original_image, pred_img, pred_masks_np, nbboxes, pred_cls, pred_conf, plot_labels=True):
  for one_mask, bbox, cls, conf in zip(pred_masks_np, nbboxes, pred_cls, pred_conf):
    if conf < 0.25:
        continue
    color = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]

    pred_img = pred_img.copy()
                             
    # Apply mask over image in color
    pred_img[one_mask] = pred_img[one_mask] * 0.5 + np.array(color, dtype=np.uint8) * 0.5
    # Draw rectangles around all found objects
    pred_img = cv2.rectangle(pred_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)

    if plot_labels:
      label = '%s %.3f' % (names[int(cls)], conf)
      t_size = cv2.getTextSize(label, 0, fontScale=0.1, thickness=1)[0]
      c2 = bbox[0] + t_size[0], bbox[1] - t_size[1] - 3
      pred_img = cv2.rectangle(pred_img, (bbox[0], bbox[1]), c2, color, -1, cv2.LINE_AA)
      pred_img = cv2.putText(pred_img, label, (bbox[0], bbox[1] - 2), 0, 0.5, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)  

  fig, ax = plt.subplots(1, 2, figsize=(pred_img.shape[0]/10, pred_img.shape[1]/10), dpi=150)

  original_image = np.moveaxis(image.cpu().numpy().squeeze(), 0, 2).astype('float32')
  original_image = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)
  
  ax[0].imshow(original_image)
  ax[0].axis("off")
  ax[1].imshow(pred_img)
  ax[1].axis("off")

In [ ]:
####################################################################################################################
# Intialize a list containing the file names of the deepsense dataset images
####################################################################################################################

In [ ]:
import os
directory = '/content/gdrive/MyDrive/Dataset/Scenario 5/camera_data'
filenames = []
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        filenames.append(filename)

print(filenames)

In [ ]:
####################################################################################################################
# The cell below will obtain the masks of the cars and their bounding boxes and store it in in a pandas dataframe
####################################################################################################################

In [ ]:

p=0
df = pd.DataFrame(columns=['Mask path', 'Bounding box coordinate 1', 'Bounding box coordinate 2', 'Bounding box coordinate 3', 'Bounding box coordinate 4', 'Number of cars'])
for _,i in enumerate(filenames):
  dummyfile_name = '/content/gdrive/MyDrive/Dataset/Scenario 5/camera_data/' + i
  output, image = run_inference(dummyfile_name)
  inf_out = output['test']
  attn = output['attn']
  bases = output['bases']
  sem_output = output['sem']

  bases = torch.cat([bases, sem_output], dim=1)
  nb, _, height, width = image.shape
  names = model.names
  pooler_scale = model.pooler_scale

  pooler = ROIPooler(output_size=hyp['mask_resolution'], 
                    scales=(pooler_scale,), 
                    sampling_ratio=1, 
                    pooler_type='ROIAlignV2', 
                    canonical_level=2)
                    

  output, output_mask, _, _, _ = non_max_suppression_mask_conf(inf_out, 
                                                              attn, 
                                                              bases, 
                                                              pooler, 
                                                              hyp, 
                                                              conf_thres=0.40, 
                                                              iou_thres=0.65, 
                                                              merge=False, 
                                                              mask_iou=None)  
  

  pred, pred_masks = output[0], output_mask[0]
  base = bases[0]
  bboxes = Boxes(pred[:, :4])

  original_pred_masks = pred_masks.view(-1, 
                                        hyp['mask_resolution'], 
                                        hyp['mask_resolution'])

  pred_masks = retry_if_cuda_oom(paste_masks_in_image)(original_pred_masks, 
                                                      bboxes, 
                                                      (height, width), 
                                                      threshold=0.5)
                                                      
  # Detach Tensors from the device, send to the CPU and turn into NumPy arrays
  pred_masks_np = pred_masks.detach().cpu().numpy()
  pred_cls = pred[:, 5].detach().cpu().numpy()
  pred_conf = pred[:, 4].detach().cpu().numpy()
  nimg = image[0].permute(1, 2, 0) * 255
  nimg = nimg.cpu().numpy().astype(np.uint8)
  nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
  nbboxes = bboxes.tensor.detach().cpu().numpy().astype(np.int)

  num_cars = 0
  o= 0
  final_arr = np.zeros((540,960))
  for one_mask, bbox, cls, conf in zip(pred_masks_np, nbboxes, pred_cls, pred_conf):
    label = '%s %.3f' % (names[int(cls)], conf)
    if "car" in label and conf>0.5:
      final_arr = np.logical_or(final_arr, pred_masks_np[o][18:540+18,0:960])
      o=o+1
      num_cars =num_cars +1
      if len(bbox.shape) == 1:
        df = df.append({'Mask path': i, 'Bounding box coordinate 1':bbox[0] , 'Bounding box coordinate 2': bbox[1]+18, 'Bounding box coordinate 3':bbox[2]  , 'Bounding box coordinate 4': bbox[3]+18 , 'Number of cars': num_cars }, ignore_index=True)
      else:
        df = df.append({'Mask path': i, 'Bounding box coordinate 1':bbox[o][0] , 'Bounding box coordinate 2': bbox[o][1]+18, 'Bounding box coordinate 3':bbox[o][2]  , 'Bounding box coordinate 4': bbox[o][3]+18 , 'Number of cars': num_cars }, ignore_index=True)  

  im = Image.fromarray(final_arr)

  im.save('/content/gdrive/MyDrive/Dataset/Scenario 5/masks/' + i)    
  p=p+1
  if p%50 == 0:
    print('50 done')



In [ ]:
df

In [ ]:
df.to_csv('/content/gdrive/MyDrive/Dataset/Scenario 5/thecsvfile.csv')
